In [1]:
import pandas as pd

In [2]:
order_products = pd.concat([pd.read_csv('../data/order_products__prior.csv'),
                            pd.read_csv('../data/order_products__train.csv')])
products = pd.read_csv('../data/products.csv')
orders = pd.read_csv('../data/orders.csv')
departments = pd.read_csv('../data/departments.csv')
aisles = pd.read_csv('../data/aisles.csv')

In [3]:
df = pd.merge(order_products, products, on='product_id')
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


In [4]:
df['values'] = 1
df = df.pivot_table(values='values', index='order_id', columns='department_id').fillna(0.0).reset_index()
df = pd.merge(df, orders, on='order_id')
df.head()

,order_id,1,2,3,4,5,6,7,8,9,...,18,19,20,21,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,112108,train,4,4,10,9.0
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,202279,prior,3,5,9,8.0
2,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,205970,prior,16,5,17,12.0
3,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,178520,prior,36,1,9,7.0
4,5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,156122,prior,42,6,16,9.0


In [7]:
# most commonly purchased product categories
df[[x for x in range(1, 22)]].sum(axis=0).sort_values(ascending=False)

4     2506247.0
16    2264738.0
7     1518833.0
19    1448749.0
1     1232089.0
13    1165491.0
3      917980.0
20     802581.0
15     710721.0
9      623738.0
12     599017.0
14     547067.0
17     492427.0
11     333463.0
6      231253.0
18     184074.0
5       87794.0
21      66410.0
8       62011.0
2       36781.0
10      35125.0
dtype: float64

In [6]:
df.to_csv('../data/dataset_clean.csv', index=False)